In [1]:
#import libraries

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
business=pd.read_csv("/Users/adityodasgupta/Documents/McGill/INSY 662/yelp_business.csv")
review=pd.read_csv("/Users/adityodasgupta/Documents/McGill/INSY 662/yelp_review.csv")

In [4]:
review.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0


In [5]:
business.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",NaN,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",NaN,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",NaN,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",NaN,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...


In [6]:
#preprocess the data

business=business[business['is_open']==1]

business.drop(columns=['neighborhood','address','city','state','postal_code','latitude','longitude','review_count','is_open'],inplace=True)

business=business[business.apply(lambda x : 'restaurant' in x.categories.lower(),axis=1)]

business['name']=business['name'].apply(lambda x : str(x).lstrip('"'))

business['name']=business['name'].apply(lambda x : str(x).rstrip('"'))


In [7]:
business.head()

,business_id,name,stars,categories
4,PfOCPjBrlQAnz__NXj9h_w,Brick House Tavern + Tap,3.5,American (New);Nightlife;Bars;Sandwiches;Ameri...
5,o9eMRCWt5PkpLDE0gOPtcQ,Messina,4.0,Italian;Restaurants
14,fNMVV_ZX7CJSDWQGdOM8Nw,Showmars Government Center,3.5,Restaurants;American (Traditional)
19,Gu-xs3NIQTj3Mj2xYoN2aw,Maxim Bakery & Restaurant,3.5,French;Food;Bakeries;Restaurants
25,1K4qrnfyzKzGgJPBEcJaNQ,Chula Taberna Mexicana,3.5,Tiki Bars;Nightlife;Mexican;Restaurants;Bars


In [8]:
#handle duplicates and unnecesary rows

review.drop(columns=['date','useful','funny','cool','stars'],inplace=True)

business.drop_duplicates(inplace=True)

review.set_index('business_id',inplace=True)

In [9]:
data=business.join(review,on='business_id',how='left')

In [10]:
data.shape

(2774509, 7)

In [11]:
data.head()

,business_id,name,stars,categories,review_id,user_id,text
4,PfOCPjBrlQAnz__NXj9h_w,Brick House Tavern + Tap,3.5,American (New);Nightlife;Bars;Sandwiches;Ameri...,rnvu74aNIPn93Z9Ue11Epw,bgtr7jOlALe8ay4g7pnf4A,I've always enjoyed my time at brick house foo...
4,PfOCPjBrlQAnz__NXj9h_w,Brick House Tavern + Tap,3.5,American (New);Nightlife;Bars;Sandwiches;Ameri...,OE5dH664oSSK_-TvIl263w,ndZOHkG_FWdL4TRd9xRfNQ,1st time here. Came w my Unc bc Louies was clo...
4,PfOCPjBrlQAnz__NXj9h_w,Brick House Tavern + Tap,3.5,American (New);Nightlife;Bars;Sandwiches;Ameri...,SWeHmo8uHHWbbv94VdAMeQ,UqtE83J-6NMQNNJzpnWwEQ,Worse service ever andI use to be a server so ...
4,PfOCPjBrlQAnz__NXj9h_w,Brick House Tavern + Tap,3.5,American (New);Nightlife;Bars;Sandwiches;Ameri...,fCCL0pAgYGccUs8shHHIUA,PixAmdRa58XZ3hr8YLS36g,I am updating my review to 5-stars because I t...
4,PfOCPjBrlQAnz__NXj9h_w,Brick House Tavern + Tap,3.5,American (New);Nightlife;Bars;Sandwiches;Ameri...,-VOh0NNnIo1r24hjwSZL4Q,Hly0NEWXMxbuXa2zm_NJOQ,I enjoyed this place. I went the night the Bu...


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2774509 entries, 4 to 174558
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   business_id  object 
 1   name         object 
 2   stars        float64
 3   categories   object 
 4   review_id    object 
 5   user_id      object 
 6   text         object 
dtypes: float64(1), object(6)
memory usage: 169.3+ MB


In [13]:


df_percent = data.sample(frac=0.015)

In [14]:
#Lower Casing
df_percent["text"] = df_percent["text"].str.lower()

#Removal of Puctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df_percent["text"] = df_percent["text"].apply(lambda x: remove_punctuation(x))

#Removal of Stopwords
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df_percent["text"] = df_percent["text"].apply(lambda x: remove_stopwords(x))

In [15]:
pd.set_option('display.max_colwidth', None)

In [17]:
#remove numbers from reviews

import re

pattern = r'[0-9]'
df_percent['text1']=df_percent.text.apply(lambda x: re.sub(pattern,'',str(x)))

In [ ]:
df_percent.set_index('name', inplace=True)
indices = pd.Series(df_percent.index)

# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['text1'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)


In [60]:
def recommend(name, cosine_similarities = cosine_similarities):
    
    # Create a list to put top restaurants
    recommend_restaurant = []
    
    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]
    
    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract top 10 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:11].index)
    
    # Names of the top 10 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(data.index)[each])
    
    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['categories', 'stars'])
    
    # Create the top 10 similar restaurants with some of their columns
    for each in recommend_restaurant:
        df_new=df_new.append(df_percent[['categories','stars']].iloc[each])
    
    # Drop the same named restaurants and sort only the top 5 by the highest rating
    df_new = df_new.drop_duplicates(subset=['categories','stars'], keep=False)
    df_new = df_new.sort_values(by='stars', ascending=False).head(5)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    
    return df_new


In [68]:
recommend('NYP- Pizzeria')

TOP 5 RESTAURANTS LIKE NYP- Pizzeria WITH SIMILAR REVIEWS: 


,categories,stars
Udon Monzo,Japanese;Restaurants,4.5
Mesa Grill,New Mexican Cuisine;Mexican;Restaurants;Southern;American (New);Tex-Mex;American (Traditional),4.0
Barrio,Bars;Restaurants;Mexican;Nightlife,4.0
Flying Fig,Bars;Vegetarian;Nightlife;American (New);Restaurants;Breakfast & Brunch,4.0
JB's Lazy Pig BBQ,Barbeque;Restaurants;Food;Ice Cream & Frozen Yogurt,4.0
